<a href="https://colab.research.google.com/github/SudeepSarkar/Computer-Vision-Course/blob/main/Lecture_18_3D_Depth_from_2_views.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D geometry from pairs of images taken from different viewpoints

1. Different cameras at different locations, scene (largely) fixed (multi-view, stereo, etc.): 

>> Large scale 3D reconstruction from internet photos https://www.cs.cornell.edu/projects/bigsfm/

2. Moving camera, scene (mostly) fixed: 

>> Augmented/mixed reality, e.g. surgery. https://www.youtube.com/watch?v=loGxO3L7rFE

>> Simultaneous localization and mapping (SLAM) https://www.youtube.com/watch?v=ZR1yXFAslSk

3. Camera fixed (maybe with pan/tilt), Objects in the scene are moving (scene monitoring, e.g., wildlife, public spaces for security)

>>https://explore.org/livecams/african-wildlife/african-river-wildlife-camera

4. Moving camera(s), Largely constant scene, but with moving objects (self-driving cars)

>> Waymo data https://waymo.com/open/

>> Nuscenes data https://www.nuscenes.org/

>> Lyft data https://self-driving.lyft.com/level5/data/


# Sketch of the solution

1. Calibrate for intrinsic parameters, $\mathbf{K_j}$, for each camera. (covered earlier)
2. Establish correspondences between pairs of images based on photometric measures, such as done in SIFT. (covered earlier)
3. Estimate epipolar geometry by estimating the fundamental $(\mathbf{F})$ matrix.
4. Recover the essential matrix, $\mathbf{E}$, from $\mathbf{F}$.
5. Estimate $\mathbf{t}$ from $\mathbf{E}$.
6. Estimate $\mathbf{R}$ from $\mathbf{E}$.
7. Given $\mathbf{R}$, $\mathbf{t}$, and $\mathbf{K_j}$, we can estimate the 3D using methods described in Lectures 15 and 16. (covered earlier)

# Recall: relationship between fundamental matrix and essential matrix.

* Recall the equations for the essential matrix, fundamental matrix, and the relationship between them.

\begin{eqnarray}
\mathbf{E} & = & \mathbf{t} \times \mathbf{R} \\
\hat{\mathbf{x}}_1^T \mathbf{E} \hat{\mathbf{x}}_0 & = & 0 \\
\mathbf{\tilde{x}}_1^T \mathbf{F} \mathbf{\tilde{x}}_0 & = & 0\\
    \mathbf{F} & = & \mathbf{K_1}^{-T} \mathbf{E} \mathbf{K_0}^{-1}\\
    \mathbf{E} & = & \mathbf{K_1}^{T} \mathbf{F} \mathbf{K_0}\\
\end{eqnarray}

* \$\mathbf{\hat{x}}_j = \mathbf{K}_j^{-1} \mathbf{\tilde{x}}_j$ is the ray from the camera center to the pixel and out to the world, with respect to the *local* camera coordinates. Note that we need to know the intrinsic camera parameters.

* Note that the fundamental matrix involves the intrinsic parameters and the extrinsic parameters, whereas the essential matrix involves the extrinsic parameters. 

*  It is **not** possible to recover the extrinsic parameters just from the fundamental matrix; we have too many unknowns (4+3+3) combined to form 9 entries of the fundamental matrix. 

* **We need the essential matrix, i.e., we need the cameras' intrinsic calibration parameters.**

#3. Estimating $\mathbf{F}$

* The constraint between the corresponding points, which are specified with respect to the center of the image (camera coordinates) in the two views, is:
\begin{eqnarray}
    \begin{bmatrix}
    \tilde{x}_1 & \tilde{y}_1 & 1
    \end{bmatrix}
     \begin{bmatrix}
     e_{00} & e_{01} & e_{02} \\
     e_{10} & e_{11} & e_{12} \\
     e_{20} & e_{21} & e_{22}
     \end{bmatrix}
     \begin{bmatrix}
    \tilde{x}_0 \\  \tilde{y}_0 \\ 1
    \end{bmatrix}
\end{eqnarray}

* Multiplying the matrices, we have one equation per corresponding point pairs.

\begin{equation}
    (\tilde{x}_0 \tilde{x}_1) e_{00} + (\tilde{y}_0 \tilde{x}_1) e_{01} + \tilde{x}_1 e_{02}  +
    (\tilde{x}_0 \tilde{y}_1) e_{10} + (\tilde{y}_0 \tilde{y}_1) e_{11} + \tilde{y}_1 e_{12}  +
    \tilde{x}_0 e_{10} + \tilde{y}_0 e_{11} +  e_{12}   = 0
\end{equation}

* Given $N$ pairs of correspondences, we will have $N$ linear equations to solve.

\begin{equation}
    \begin{bmatrix}
    \vdots & \vdots &\vdots & \vdots & \vdots &\vdots & \vdots & \vdots & \vdots \\
    \vdots & \vdots &\vdots & \vdots & \vdots &\vdots & \vdots & \vdots & \vdots \\
    (\tilde{x}_0 \tilde{x}_1) & (\tilde{y}_0 \tilde{x}_1) & \tilde{x}_1 &
    (\tilde{x}_0 \tilde{y}_1) & (\tilde{y}_0 \tilde{y}_1) & \tilde{y}_1 &
    \tilde{x}_0 & \tilde{y}_0 &  1  \\
    \vdots & \vdots &\vdots & \vdots & \vdots &\vdots & \vdots & \vdots & \vdots \\
    \vdots & \vdots &\vdots & \vdots & \vdots &\vdots & \vdots & \vdots & \vdots \\
    \end{bmatrix}
    \begin{bmatrix}
 e_{00} \\ e_{01} \\ e_{02} \\
     e_{10} \\ e_{11} \\ e_{12} \\
     e_{20} \\ e_{21} \\ e_{22}
\end{bmatrix} = 0
\end{equation}

* Or equivalently, we have to solve the following linear equation. Note the uncertainty concerning the scaling of the solution. (We have seen equations like this before, and you should be able to sketch out at least three ways to solve this.)

\begin{eqnarray}
\mathbf{A}^{N\times 9} \mathbf{e}^{9\times 1} & = & 0 \\
\end{eqnarray}


---

\begin{eqnarray}
\mathbf{A}^{N\times 9} \mathbf{e}^{9\times 1} & = & 0 \\
\min || \mathbf{A} \mathbf{e}||^2  \\
\min (\mathbf{A} \mathbf{e})^T \mathbf{A} \mathbf{e} \\
\min \mathbf{e}^T (\mathbf{A}^T \mathbf{A})^{9 \times 9} \mathbf{e} 
\end{eqnarray}

* To avoid the degenerate solution, $\mathbf{e} = 0$  we add the constraint that $||\mathbf{e}|| = 1$. Note that we can recover the vector $\mathbf{e}$ only upto a scale.

* So the minimization problem, using Lagrange multipliers, can be expressed as  
\begin{eqnarray}
    \min \mathbf{e}^T (\mathbf{A}^T \mathbf{A}) \mathbf{e} + \lambda (1 - \mathbf{e}^T \mathbf{e})
\end{eqnarray}

* Taking the derivative of the above with respect to $\mathbf{e}$ and setting it to zero, we have

\begin{eqnarray}
    (\mathbf{A}^T \mathbf{A}) \mathbf{e}  = \lambda \mathbf{e}
\end{eqnarray}

* Solution is the eigenvector corresponding to the smallest eigenvalue of $\mathbf{A}^T \mathbf{A}$.

----


#4. Recover essential matrix from the fundamental matrix and known intrinsic parameters.
* Rearrange this estimate of $\mathbf{e}$ into a 3 by 3 matrix $\mathbf{E}$

* Compute the essential matrix using: $\mathbf{E}  =  \mathbf{K_1}^{T} \mathbf{F} \mathbf{K_0}$


#5.  Estimating $\mathbf{\hat{t}}$ from $\mathbf{E}$
* Note the columns of $\mathbf{E}$ are all orthogonal to $\mathbf{t}$ (WHY?), so **$\mathbf{t}^T\mathbf{E} = 0 $**. This means that one of the singular values will be zero. 



* The translation (direction only) solution is the left vector corresponding to the smallest singular value of the SVD decomposition. 

For a refresher on SVD, see the appendix of the book or 

>> https://web.mit.edu/be.400/www/SVD/Singular_Value_Decomposition.htm

* (Also, recall the connection of epipoles to the translation vector. The translation vector that we seek takes a coordinate from the left camera into the right camera. This vector is defined w.r.t the right camera - should be in the opposite direction of the right epipole.)

* Let the SVD of $\mathbf{E}$ be denoted
\begin{equation}
\mathbf{E} = \mathbf{U} \Lambda \mathbf{V}^T =
    \begin{bmatrix}
    \mathbf{u}_0 & \mathbf{u}_1 & \mathbf{u}_2
    \end{bmatrix}
    \begin{bmatrix}
    \sigma_0 & & \\
    & \sigma_1 & \\
    & & \sigma_2 
    \end{bmatrix}
    \begin{bmatrix}
    \mathbf{v}_0^T \\ \mathbf{v}_1^T \\ \mathbf{v}_2^T
    \end{bmatrix}
\end{equation}

>>>>where $\sigma_0 > \sigma_1 > \sigma_2 $. 

* Ideally, $\sigma_2$ should be zero. So for a correct estimate of the essential matrix, we can set $\sigma_2=0$. And, $\mathbf{\hat{t}} = \mathbf{u}_2 =  $, the unit vector along the translation. 

\begin{equation}
\mathbf{E'} = \mathbf{U} \Lambda \mathbf{V}^T =
    \begin{bmatrix}
    \mathbf{u}_0 & \mathbf{u}_1 & \mathbf{\hat{t}}
    \end{bmatrix}
    \begin{bmatrix}
    \sigma_0 & & \\
    & \sigma_1 & \\
    & & 0 
    \end{bmatrix}
    \begin{bmatrix}
    \mathbf{v}_0^T \\ \mathbf{v}_1^T \\ \mathbf{v}_2^T
    \end{bmatrix}
\end{equation}

* **We can only recover the direction of the translation, not the magnitude. Impossible to recover distance between cameras without a real-world dimension. If we scale the world, we will get the same pairs of images.**

#6. Estimating $\mathbf{R}$ from $\mathbf{E}$

* We will solve for the rotation matrix by drawing a parallel between the SVD of $\mathbf{E}$  and the SVD of the product of a skew-symmetric matrix and orthonormal rotation matrix. 

* Ideally, in a noise-free world we can say one more thing about the singular values:  $\sigma_0 =  \sigma_1 = 1$. This is because (from Wikipedia): "Not every arbitrary $3\times 3$ matrix can be an essential matrix for some stereo cameras. To see this, notice that the essential matrix is defined as the matrix product of one rotation matrix and one skew-symmetric matrix, both $3\times 3$. **Any skew-symmetric matrix has two singular values, which are equal and another which is zero.** The multiplication of the rotation matrix does not change singular values. This means that the essential matrix also has two equal singular values and one which is zero. The properties described here are sometimes referred to as internal constraints of the essential matrix."

* Using this, we can further correct the essential matrix to

\begin{equation}
\mathbf{E''} = \mathbf{U} \Lambda \mathbf{V}^T =
    \begin{bmatrix}
    \mathbf{u}_0 & \mathbf{u}_1 & \mathbf{\hat{t}}
    \end{bmatrix}
    \begin{bmatrix}
    1& & \\
    & 1 & \\
    & & 0 
    \end{bmatrix}
    \begin{bmatrix}
    \mathbf{v}_0^T \\ \mathbf{v}_1^T \\ \mathbf{v}_2^T
    \end{bmatrix}
\end{equation}

* Next, let us consider the skew symmetric matrix $\mathbf{S}$. The skew symmetric property is: $\mathbf{S}^T = - \mathbf{S}$. The SVD of the skew symmetric matrix will have the following properties:
>> 1. The left singular vectors (i.e. the eigenvectors of $\mathbf{S} \mathbf{S}^T = - \mathbf{S}\mathbf{S}$) will be the same as the right singular vectors (i.e. the eignvectors of $\mathbf{S} \mathbf{S}^T = - \mathbf{S} \mathbf{S}$).
>> 2. The singular values will be square of the eigenvalues of $\mathbf{S}$. The eigenvalues of $\mathbf{S}$ can be shown to be $+ i, -i, 0$. We can derive this using $\det(\mathbf{S} - \lambda \mathbf{I}) = 0$. 

* So, the skew-symmetric matrix's singular values are $1, 1,$ and $0$. 

* The skew-symmetric matrix composed from the translation estimate be denoted by $[\mathbf{\hat{t}}]_{\times}$. By performing matrix multiplication, it is easy to show that $\hat{t}$ is both a left and right singular vector.

\begin{eqnarray}
\begin{bmatrix}
    0 & -\hat{t}_z & \hat{t}_y \\ \hat{t}_z & 0 & -\hat{t}_x \\ -\hat{t}_y & \hat{t}_x & 0
\end{bmatrix} 
\begin{bmatrix}
    \hat{t}_x \\ \hat{t}_y\\ \hat{t}_z
\end{bmatrix}  & = & 0 \\
\begin{bmatrix}
    \hat{t}_x & \hat{t}_y & \hat{t}_z
\end{bmatrix} 
\begin{bmatrix}
    0 & -\hat{t}_z & \hat{t}_y \\ \hat{t}_z & 0 & -\hat{t}_x \\ -\hat{t}_y & \hat{t}_x & 0
\end{bmatrix} 
 & = & 0 \\
\end{eqnarray}

* The other two singular vectors can be switched with each other (as their corresponding singular values are the same), and we can flip the signs of the vectors in the SVD without changing the decomposition.

* Thus, the SVD of the skew-symmetric cross-product matrix, $[\mathbf{\hat{t}}]_{\times}$,  can be expressed as

\begin{equation}
    [\mathbf{\hat{t}}]_{\times} = 
    \begin{bmatrix} \mathbf{s}_0 & \mathbf{s}_1 & \mathbf{t}  \end{bmatrix}
    \begin{bmatrix} 1 & & \\ & 1 & \\ & & 0 \end{bmatrix}
    \begin{bmatrix} 0 & -1 & \\ 1 & 0 & \\ & & 1 \end{bmatrix}
    \begin{bmatrix} \mathbf{s}_0^T \\ \mathbf{s}_1^T \\ \mathbf{t}^T \end{bmatrix}
\end{equation}

* Using this decomposition of the cross-product matrix, the essential matrix can also be expressed as

\begin{equation}
\mathbf{E} = \mathbf{\hat{t}} \times \mathbf{R} = 
\begin{bmatrix} \mathbf{s}_0 & \mathbf{s}_1 & \mathbf{\hat{t}} \end{bmatrix}
    \begin{bmatrix} 1 & & \\ & 1 & \\ & & 0 \end{bmatrix}
    \begin{bmatrix} 0 & -1 & \\ 1 & 0 & \\ & & 1 \end{bmatrix}
    \begin{bmatrix} \mathbf{s}_0^T \\ \mathbf{s}_1^T \\ \mathbf{\hat{t}}^T \end{bmatrix}
    \begin{bmatrix}  & & \\ & \mathbf{R} & \\ & & \end{bmatrix}
\end{equation}

* Compare this with the SVD of the $\mathbf{E} = \mathbf{U} \Lambda \mathbf{V}^T $ and choose $\mathbf{s}_0 = \mathbf{u}_0$ and $\mathbf{s}_1 = \mathbf{u}_1$. Then we can see the correspondence between the rotation matrix and rest of the expression.

\begin{eqnarray}
    \mathbf{R}_{90^0} \mathbf{U}^T \mathbf{R} & = & \mathbf{V}^T \\
    \mathbf{R} & = & \mathbf{U}\mathbf{R}_{90^0}^T \mathbf{V}^T
\end{eqnarray}

* Possible four solutions of rotation matrix: the $\mathbf{R}_{90^0}$ can also be a rotation the other way, $\mathbf{R}_{-90^0}$ and the signs of the eigenvectors can be flipped in the SVD without change the decomposition.

\begin{equation}
    \mathbf{R} = \pm \mathbf{U}\mathbf{R}_{\pm 90^0}^T \mathbf{V}^T
\end{equation}

>> where

\begin{equation}
   \mathbf{R}_{90^0} = 
   \begin{bmatrix} 0 & -1 & \\ 1 & 0 & \\ & & 1 \end{bmatrix}
\end{equation}

>> and

\begin{equation}
   \mathbf{R}_{-90^0} = 
   \begin{bmatrix} 0 & 1 & \\ -1 & 0 & \\ & & 1 \end{bmatrix}
\end{equation}

Keep the two estimates of $\mathbf{R}$ whose determinant $\det{\mathbf{R}}=1$. Pair them with two possibilities of the translation vector $\pm \mathbf{t}$ to arrive at 4 possible combinations. 

* Use these estimates of the camera transformation to estimate the coordinates of the corresponding pixel pairs. Pick the camera transformation for which we get the most positive $z$-coordinates (a condition is also known as **chirality**).

>>> https://en.wikipedia.org/wiki/Essential_matrix

